# ECA Urban extents

Calculate urban extents using GHS-Pop for all of ECA; attribute with nighttime lights values

In [ ]:
import sys
import os
import json
import boto3
import multiprocessing
import rasterio

import pandas as pd
import geopandas as gpd

from botocore.config import Config
from botocore import UNSIGNED
from shapely.geometry import Point

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")
import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.ntlMisc as ntl
from GOSTRocks.misc import tPrint

sys.path.append("../../../src")
import GOST_Urban.country_helper as country_helper

%load_ext autoreload
%autoreload 2

# read in local important parameters
local_json = "/home/wb411133/Code/urbanParameters.json"
with open(local_json, "r") as inJ:
    important_vars = json.load(inJ)

s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))

In [ ]:
global_population_ghs_file = "/home/public/Data/GLOBAL/Population/GHS/2022_1km/GHS_POP_E2020_GLOBE_R2022A_54009_1000_V1_0.tif"
global_admin = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
global_ghsl_folder = "/home/public/Data/GLOBAL/GHSL/v2022/"

output_folder = "/home/wb411133/projects/KAZ_SCADR_Urbanization/DATA/ECA_Extents"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

ntl_files = ntl.aws_search_ntl()
ghsl_files = [
    os.path.join(global_ghsl_folder, x)
    for x in os.listdir(global_ghsl_folder)
    if x.endswith(".tif")
]
ghsl_files.sort()

In [ ]:
inR = rasterio.open(global_population_ghs_file)

allAdmin = gpd.read_file(global_admin)
inAdmin = allAdmin.loc[
    (allAdmin["Region"] == "Europe & Central Asia")
    | (allAdmin["ISO3"].isin(["RUS", "ROU", "HRV"]))
]
inAdmin = inAdmin.to_crs(inR.crs)

# Calculate urban with helper object

In [ ]:
all_args = []
for idx, row in inAdmin.iterrows():
    iso3 = row["ISO3"]
    tPrint(f"*********STARTING {iso3}")
    sel_country = gpd.GeoDataFrame(inAdmin.loc[inAdmin["ISO3"] == iso3], crs=inR.crs)
    sel_country["geometry"] = sel_country["geometry"].apply(lambda x: x.buffer(0))
    cur_folder = os.path.join(output_folder, iso3)
    if not os.path.exists(cur_folder):
        os.makedirs(cur_folder)
    pop_file = os.path.join(cur_folder, f"{iso3}_ghs_pop_2020.tif")
    if not os.path.exists(pop_file):
        rMisc.clipRaster(inR, sel_country, pop_file)
    inP = rasterio.open(pop_file)
    if iso3 == "HRV":
        all_args.append(
            [iso3, sel_country, cur_folder, pop_file, ntl_files, ghsl_files]
        )

In [ ]:
def run_extractor(iso3, sel_country, cur_folder, inP, ntl_files, ghsl_files):
    extractor = country_helper.urban_country(iso3, sel_country, cur_folder, inP)
    # extractor.delete_urban_data()
    extractor.calculate_urban_extents()
    extractor.summarize_ntl(ntl_files=ntl_files)
    extractor.summarize_ghsl(ghsl_files, clip_raster=True, binary_calc=True)

In [ ]:
# run a single country
run_extractor(*all_args[0])

In [ ]:
with multiprocessing.Pool(len(all_args)) as pool:
    pool.starmap(run_extractor, all_args)

# Merge data

In [ ]:
hd_files = []
center_files = []
for root, folders, files in os.walk(output_folder):
    for f in files:
        if f.endswith("extents.geojson"):
            center_files.append(os.path.join(root, f))
        if f.endswith("extents_hd.geojson"):
            hd_files.append(os.path.join(root, f))

In [ ]:
all_res = []

for cFile in center_files:
    curD = gpd.read_file(cFile)
    iso3 = os.path.basename(cFile)[:3]
    curD["ISO3"] = iso3
    all_res.append(curD)

final_center = pd.concat(all_res)

In [ ]:
all_res = []

for cFile in hd_files:
    curD = gpd.read_file(cFile)
    iso3 = os.path.basename(cFile)[:3]
    curD["ISO3"] = iso3
    all_res.append(curD)

hd_center = pd.concat(all_res)

In [ ]:
# match cities to centers
inCities = pd.read_csv(
    "/home/wb411133/projects/KAZ_SCADR_Urbanization/DATA/CITIES/worldcities.csv"
)
geoms = [Point(x) for x in zip(inCities["lng"], inCities["lat"])]
inCities = gpd.GeoDataFrame(inCities, geometry=geoms, crs=4326)
inCities.to_file(
    "/home/wb411133/projects/KAZ_SCADR_Urbanization/DATA/CITIES/worldcities.geojson",
    driver="GeoJSON",
)
inCities.head()

In [ ]:
final_center.head()

In [ ]:
final_center["wCity"] = ""
final_center.reset_index(inplace=True)

for idx, row in final_center.iterrows():
    try:
        sel_city = inCities.loc[inCities.intersects(row["geometry"])]
    except:
        sel_city = inCities.loc[inCities.intersects(row["geometry"].buffer(0))]
    if sel_city.shape[0] > 0:
        final_center.loc[idx, "wCity"] = sel_city["city"].iloc[0]

In [ ]:
final_center.to_file(
    os.path.join(output_folder, "all_urban_centers.geojson"), driver="GeoJSON"
)

In [ ]:
hd_center["wCity"] = ""
hd_center.reset_index(inplace=True)
for idx, row in hd_center.iterrows():
    try:
        sel_city = inCities.loc[inCities.intersects(row["geometry"])]
    except:
        sel_city = inCities.loc[inCities.intersects(row["geometry"].buffer(0))]
    if sel_city.shape[0] > 0:
        hd_center.loc[idx, "wCity"] = sel_city["city"].iloc[0]
        break

In [ ]:
hd_center

In [ ]:
hd_center.to_file(
    os.path.join(output_folder, "all_hd_urban_centers.geojson"), driver="GeoJSON"
)

# Delete GHSL files

In [ ]:
bad_files = []
for root, dirs, files in os.walk(output_folder):
    for f in files:
        if f.endswith("100_V1_0.tif"):
            bad_files.append(os.path.join(root, f))

bad_files

In [ ]:
for f in bad_files:
    os.remove(f)